출처 : https://bab2min.tistory.com/570  
출처 : https://wiserloner.tistory.com/932

In [1]:
!pip install networkx
!pip install konlpy

     |████████████████████████████████| 19.4MB 164kB/s 
     |████████████████████████████████| 460kB 44.7MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran
import networkx
import re
import os
import math

### **RawTextReader**
file path를 읽어, Iterate할 때 문장 단위로 반환해주는 클래스


In [56]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s


### **TextRank**

문서 내 문장들의 TR을 구하는 클래스

**```loadSents```** 

문장을 읽어 품사 태깅 값 저장한 후, 문장 간 유사도를 비교하여 일정 이상 유사할 경우 ```dictBiCount[i,j]```에 저장하는 함수이다.    

- ```dictCount``` : 문장 저장  
- ```sentSet``` : 문장의 품사 태깅한 값 저장 (명사, 동사, 형용사만 추출)  
- ```dictBiCount``` : 현재 문장 i와 이전 문장 j의 유사도를 비교하여, threshold를 넘을 경우 ```dictBiCount[i,j]```에 유사도 값 저장    
  

유사도는 각 문장 sentSet 간의 교집합 원소 개수를 n이라 했을 때, 아래 식을 계산한 값이다.  
아래 유사도는 자카드 유사도 계산식에 따라 계산되었으며, 0에서 1 사이 값을 가진다.  
```
similarity = n / float(len(A) + len(B) - n) / (math.log(len(A)+1) * math.log(len(B)+1))
```
  
**```build```**
 
문장의 개수만큼 노드를 생성하고, i 노드와 j 노드 사이 간선에 아래 식을 계산한 값을 부여한다.  

```
weight = dictBiCount[i,j]*self.coef + (1-self.coef) # self.coef = 1.0 (미지정)
```

   
**```rank```**  
생성한 그래프를 바탕으로 각 노드의 ```pagerank```를 구한다.  


**```summarize```**  
입력 값 ratio 값에 따라 ```int(문장 개수 * ratio)``` 만큼 중요 문장을 이어붙여 반환한다. 


### **품사 태깅 (POS Tagging, part-of-speech tagging)**

각 단어의 품사를 태깅하는 프로세스.

Komoran는 형태소 분석을 지원하며, 이를 사용하여 품사 태깅 진행  
```NNG``` : 일반 명사   
```NNP``` : 고유 명사  
```VV``` : 동사  
```VA``` : 형용사  

articles/Pretty-Data 경로 내 있는 데이터는, 문장단위 구분만을 수행한 원본 데이터이며 아래 ```sent```와 같다.

In [20]:

sent = "청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다"
tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

tokenizer = lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))
fil = tokenizer(sent)

set(filter(None, fil))

{('경무', 'NNP'),
 ('경위', 'NNG'),
 ('경찰', 'NNG'),
 ('경찰대', 'NNG'),
 ('계장', 'NNG'),
 ('고와', 'NNP'),
 ('기획', 'NNG'),
 ('송해', 'NNP'),
 ('역임', 'NNG'),
 ('영', 'NNP'),
 ('예산', 'NNP'),
 ('입문', 'NNG'),
 ('졸업', 'NNG'),
 ('청', 'NNP'),
 ('청주', 'NNP'),
 ('출신', 'NNG'),
 ('충북', 'NNP'),
 ('홍보', 'NNG')}

articles/Preprocessed-Data 경로 내 있는 데이터는, 문장단위 구분, 중복 공백 및 특수문자 삭제, 불용어 삭제를 수행한 전처리 된 데이터이며 아래 출력과 같다.


TF-IDF를 이용하여 문장 간 유사도를 구하기 위해 아래 전처리 된 데이터를 사용한다.  
 ```Document``` 클래스를 이용해 원본과 전처리 데이터를 매핑하여 출력한다.

In [18]:
fil = tokenizer(sent)
s= ' '.join([noun for (noun, pos) in list(fil)])
s

'청주 출신 청주 고와 경찰대 졸업 송해 영 계장 경위 경찰 입문 충북 청 기획 예산 계장 경무 계장 홍보 계장 역임'

In [58]:
class Document:
    def __init__(self, originSentenceIter, procSentenceIter):
        self.originSents = list(filter(None, originSentenceIter))
        self.procSents = list(filter(None, procSentenceIter))
    
    def getOriginSet(self):
        return self.originSents

    def getSentsZip(self):
        return zip(self.originSents, self.procSents)

In [59]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}

        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = {}

    
    def loadSents(self, document, tokenizer, similarity = 'jaccard'):
        import math
        def jaccard_similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))

        def tfidf_cosine_similarity(i, j):
            from sklearn.metrics.pairwise import cosine_similarity
            return cosine_similarity(self.tfidf_matrix[i-1:i], self.tfidf_matrix[j-1:j])[0,0]
 
        sentSet = []
        for origin, proc in document.getSentsZip():
            tagged = set(filter(None, tokenizer(proc)))
            if len(tagged) < 2: continue
            self.dictCount[len(self.dictCount)] = origin
            sentSet.append(tagged)
        
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(document.getOriginSet())

        if similarity is not 'jaccard' :
            for i in range(1, len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    s = tfidf_cosine_similarity(i, j)

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
        else: 
            for i in range(len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    s = jaccard_similarity(sentSet[i], sentSet[j])

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
    
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

기사 파일 경로 지정



> ```articles/Origin-Data``` :  크롤링한 원본 데이터, (기사 제목, 본문, 언론사를 포함한다.)  
> ```articles/Preprocessed-Data``` :  전처리를 마친 데이터 (중복 공백, 특수문자, 불용어 제거)  
> ```articles/Pretty-Data``` :  원본 데이터에서 본문을 문장 단위로 구분한 데이터  
> ```artitlces/StopWordList```:  불용어 리스트



In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [43]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/Text-preprocessing-Data/articles"
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [44]:
media_list = os.listdir(PREPROCESSED_PATH)

origin_article_list = os.listdir(os.path.join(PRETTY_PATH, media_list[0]))
proc_article_list = os.listdir(os.path.join(PREPROCESSED_PATH, media_list[0]))

origin_article_path = os.path.join(os.path.join(PRETTY_PATH, media_list[0]), origin_article_list[0])
proc_article_path = os.path.join(os.path.join(PREPROCESSED_PATH, media_list[0]), proc_article_list[0])

불용어로 StopWordList.txt 파일 내 있는 용어들을 사용하였으며 이는 품사 태깅에서 제외될 항목이다.  
rank 함수를 통해 얻은 각 문서의 TR값에 따라 정렬할 경우, 중요한 순서대로 나열된다.  



In [53]:
def getStopWord(filename):
    swords = []
    tagger = Komoran()
    with open(filename, 'r', encoding='utf-8') as f:
        swords = f.readlines()
        swords = [tag for sword in swords for tag in tagger.pos(sword.strip()) if tag[1] in ('NNG', 'NNP', 'VV', 'VA')]

    return swords

In [54]:
# stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
stopword = getStopWord(SWORDS_PATH)

자카드 유사도 계산식을 사용하여 문장 간 유사도를 구한 경우

*출력된 문장은 원본 문장이다.*

In [65]:
tr = TextRank()
tagger = Komoran()

tr.loadSents(Document(RawTextReader(origin_article_path), RawTextReader(proc_article_path)),
             lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))

tr.build()

ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

1	0.09950128890476066	 경찰이 범인을 잡고보니 현직 초급경찰간부였다.
14	0.09329151425490463	 경찰은 이 차를 주목했다.
17	0.08038877229382121	경찰은 그를 상대로 조사중이다.
6	0.07021955295302305	경찰이 도난범 경찰을 검거한 특이한 상황이 발생했다.
16	0.05869681558432278	 경찰은 차량운전자와 소유자를 특정인으로 지목했다.
18	0.05204127590292003	 광주남부경찰서는 “그가 범행을 인정했다”고 밝혔다.
5	0.05112237770073722	 경찰은 신상을 파악한 뒤 병원을 급습하여 법인을 체포했다.
3	0.04660735987973594	 도난범을 잡은 것은 지난 6일 밤 11시쯤이었다.
24	0.04424778761061947	[조홍복 기자 powerbok@chosun.
25	0.04424778761061947	com] [권경안 기자 gakwon@chosun.
10	0.04161764111471065	범행 일시는 지난달 18일 오전 4시.
22	0.03990352013673729	현직 경찰로 도둑이 된 그에 대해 수사경찰은 “무관용의 원칙을 바탕으로 엄중하게 대응하겠다”며 “조사를 마치면 구속영장을 신청하겠다”고 말했다.
8	0.03569915620000391	 초급간부였다.
2	0.03435410892394138	금은방 주인의 도난신고를 받은 광주남부경찰서는 바로 전담팀을 조직, 수사에 착수했다.
0	0.03020433519292796	지난 달 18일 새벽 광주광역시 남구 월산동의 한 금은방에서 2500만원 상당의 귀금속이 도난당했다.
20	0.02936647524126016	 남부서는 “그가 다액의 채무를 범행동기로 진술하고 있고, 관련내용을 명확히 확인중에 있다”고 말했다.
15	0.02837873119773143	 관내 도로망 곳곳의 CCTV 등을 확인하며 차량의 꼬리를 따라갔다.
9	0.026248222605615174	 광주서부경찰서 관내 한

TFIDF를 이용하여 코사인 유사도 계산식을 사용하여 문장 간 유사도를 구한 경우

In [60]:
tr = TextRank()
tagger = Komoran()

tr.loadSents(Document(RawTextReader(origin_article_path), RawTextReader(proc_article_path)), 
             lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)), similarity='tfidf')

tr.build()
ranks = tr.rank()

for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

2	0.07685516612631652	금은방 주인의 도난신고를 받은 광주남부경찰서는 바로 전담팀을 조직, 수사에 착수했다.
19	0.07412552985469067	 피해품은 모두 회수했다.
1	0.0727852959195138	 경찰이 범인을 잡고보니 현직 초급경찰간부였다.
21	0.07202308105713578	 그는 억대의 도박빚을 지고 있을 것으로 알려졌다.
10	0.06238593534402684	범행 일시는 지난달 18일 오전 4시.
15	0.05663492716553529	 관내 도로망 곳곳의 CCTV 등을 확인하며 차량의 꼬리를 따라갔다.
18	0.05180045696426526	 광주남부경찰서는 “그가 범행을 인정했다”고 밝혔다.
8	0.05070504146146192	 초급간부였다.
12	0.04987531172069826	 준비해놓은 도구로 금은방의 문을 부수고 침임했다.
22	0.04987531172069826	현직 경찰로 도둑이 된 그에 대해 수사경찰은 “무관용의 원칙을 바탕으로 엄중하게 대응하겠다”며 “조사를 마치면 구속영장을 신청하겠다”고 말했다.
17	0.048134986508636866	경찰은 그를 상대로 조사중이다.
23	0.04789608074891488	 그에게는 읿반절도보다 무거운 특수절도 혐의가 적용되고 있다.
6	0.04293087624435567	경찰이 도난범 경찰을 검거한 특이한 상황이 발생했다.
4	0.038420319621290486	 광주시내 한 병원이었다.
11	0.038420319621290486	 그는 마스크와 모자를 쓰고 있었다.
7	0.03187471368928496	 계급은 경위로 현직 경찰관이었다.
3	0.031562073911949914	 도난범을 잡은 것은 지난 6일 밤 11시쯤이었다.
16	0.025781129385744915	 경찰은 차량운전자와 소유자를 특정인으로 지목했다.
24	0.02554436562745622	[조홍복 기자 powerbok@chosun.
0	0.00748129675

### **TF-IDF를 이용한 문장 유사도 계산** 

특정 문서 내에서 단어 빈도가 높을 수록, 그리고 전체 문서들 중 그 단어를 포함한 문서가 적을 수록 TF-IDF값이 높아진다.

- TF : 특정 단어 t가 문서 d 내에서 얼마나 자주 등장하는 지 나타내는 지표
- IDF : 특정 단어가 등장한 문서의 수를 나타내는 지표인 DF에 반비례하는 수 

*|D|는 전체 문서의 수를 의미한다.*

```
tfidf(t, d, D) = tf(t, d) * idf(t, D)
```


In [78]:
threshold = 0.005
dictCount = {}
dictBiCount = {}

sentenceIter = RawTextReader(proc_article_path)
tokenizer = lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))

if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
sentSet = []

In [79]:
sents = set(filter(None, sentenceIter))
sents

{'경안 기자',
 '경찰 도난 범 경찰 검거 상황 발생',
 '경찰 범인 잡 현직 초급 경찰 간부',
 '경찰 상대로 조사',
 '경찰 신상 파악 뒤 병원 급 습 법인 체포',
 '경찰 차 주목',
 '경찰 차량 운전자 소유자 특정인 지목',
 '계급 경위 현직 경찰관',
 '관내 도로망 곳곳 확인 차량 꼬리 따라가',
 '광주남부경찰서 범행 인정 밝히',
 '광주서부경찰서 관내 파출소 근무 도둑 잡 도둑 체포 순간',
 '광주시 병원',
 '금은방 주인 도난 신고 받 광주남부경찰서 전담 팀 조직 수사 착수',
 '남 부서 다 액 채무 범행 동기 진술 있 관련 내용 확인 있 말',
 '도난 범 잡 지나 밤',
 '도박 빚 있 알리',
 '마스크 모자 쓰',
 '무겁 특수 절도 혐의 적용 있',
 '범행 일시 지난달 오전',
 '준비 도구 금은방 문 부수 침 임하',
 '지나 달 새벽 광주광역시 남구 월산동 금은방 만원 상당 귀금속 도난',
 '초급 간부',
 '침입자 번호판 가리 차량 타 도망가',
 '피해 품 회수',
 '현직 경찰 도둑 되 대하 수사 경찰 관용 원칙 바탕 대응 조사 마치 구속영장 신청 말',
 '홍복 기자'}

In [80]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sents)
idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf)))

{'가리': 3.6026896854443837, '간부': 3.1972245773362196, '검거': 3.6026896854443837, '경안': 3.6026896854443837, '경위': 3.6026896854443837, '경찰': 2.216395324324493, '경찰관': 3.6026896854443837, '계급': 3.6026896854443837, '곳곳': 3.6026896854443837, '관내': 3.1972245773362196, '관련': 3.6026896854443837, '관용': 3.6026896854443837, '광주광역시': 3.6026896854443837, '광주남부경찰서': 3.1972245773362196, '광주서부경찰서': 3.6026896854443837, '광주시': 3.6026896854443837, '구속영장': 3.6026896854443837, '귀금속': 3.6026896854443837, '근무': 3.6026896854443837, '금은방': 2.9095425048844383, '기자': 3.1972245773362196, '꼬리': 3.6026896854443837, '남구': 3.6026896854443837, '내용': 3.6026896854443837, '대응': 3.6026896854443837, '대하': 3.6026896854443837, '도구': 3.6026896854443837, '도난': 2.6863989535702286, '도둑': 3.1972245773362196, '도로망': 3.6026896854443837, '도망가': 3.6026896854443837, '도박': 3.6026896854443837, '동기': 3.6026896854443837, '따라가': 3.6026896854443837, '마스크': 3.6026896854443837, '마치': 3.6026896854443837, '만원': 3.6026896854443837, '모자': 3.6026896

In [72]:
test_list= os.listdir(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]))
test_path = os.path.join(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]), test_list[0])

with open(test_path, 'r', encoding='utf-8') as f:
    title = f.readline()[:-1]
    content = f.readline()[:-1]

#### **Gensim 패키지를 이용한 TF_IDF 추출**

In [85]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [83]:
test_list= os.listdir(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]))
test_path = os.path.join(os.path.join(ORIGIN_PATH, os.listdir(ORIGIN_PATH)[0]), test_list[0])

## 테스트할 원본 데이터
with open(test_path, 'r', encoding='utf-8') as f:
    title = f.readline()[:-1]
    content = f.readline()[:-1]
    print(title, content)

충북 경찰 4년 연속 총경 2명 배출 (청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다. 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다. 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다. 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다. 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.reason@news1.kr© 뉴스1코리아(), 무단 전재 및 재배포 금지


In [86]:
import gensim
import pprint
from gensim import corpora
from gensim import models
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
import numpy as np

doc_list = sent_tokenize(content)

doc_tokenized = [simple_preprocess(doc) for doc in doc_list]
dictionary = corpora.Dictionary()

# token 빈도
BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]

for doc in BoW_corpus:
   print([[dictionary[id], freq] for id, freq in doc])
tfidf = models.TfidfModel(BoW_corpus, smartirs='ntc')

print('\nTF-IDF')
for doc in tfidf[BoW_corpus]:
   print([[dictionary[id], np.around(freq,decimals=2)] for id, freq in doc])

[['경찰에서', 1], ['기자', 1], ['나왔다', 1], ['뉴스', 1], ['명의', 1], ['승진자가', 1], ['연속', 1], ['조준영', 1], ['청주', 1], ['총경', 1], ['충북', 1]]
[['경정', 1], ['경찰대', 1], ['경찰청은', 1], ['공채', 1], ['김경태', 1], ['내정했다', 1], ['단행한', 1], ['명을', 1], ['송해영', 1], ['승진', 1], ['여성보호계장', 1], ['인사에서', 1], ['일반', 1], ['총경으로', 1], ['충북경찰청', 1], ['충북청', 1], ['포함한', 1], ['홍보계장', 1]]
[['청주', 1], ['김경태', 1], ['경찰에', 1], ['계장은', 1], ['괴산', 1], ['운호고와', 1], ['입문했다', 1], ['졸업한', 1], ['청주대학교를', 1], ['출신인', 1]]
[['청주', 1], ['경비교통과장', 1], ['등을', 1], ['보안수사대장', 1], ['생활안전과장과', 1], ['생활질서계장', 1], ['이후', 1], ['지냈다', 1], ['충청', 1], ['흥덕경찰서', 1]]
[['뉴스', 1], ['청주', 1], ['송해영', 1], ['충북청', 1], ['홍보계장', 1], ['경찰에', 1], ['계장은', 1], ['졸업한', 1], ['등을', 1], ['kr', 1], ['news', 1], ['reason', 1], ['경무계장', 1], ['경위로', 1], ['경찰대를', 1], ['금지', 1], ['기획예산계장', 1], ['무단', 1], ['역임했다', 1], ['입문해', 1], ['재배포', 1], ['전재', 1], ['청주고와', 1], ['출신으로', 1], ['코리아', 1]]

TF-IDF
[['경찰에서', 0.33], ['기자', 0.33], ['나왔다', 0.33], ['뉴스', 0.19], ['명의', 0.33], ['승진자

#### **sklearn의 TfidfVectorizer를 이용한 TF_IDF 추출**



In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentenceIter = RawTextReader(proc_article_path)
docs = list(sentenceIter)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
tfidf_vectorizer.vocabulary_


{'가리': 0,
 '간부': 1,
 '검거': 2,
 '경안': 3,
 '경위': 4,
 '경찰': 5,
 '경찰관': 6,
 '계급': 7,
 '곳곳': 8,
 '관내': 9,
 '관련': 10,
 '관용': 11,
 '광주광역시': 12,
 '광주남부경찰서': 13,
 '광주서부경찰서': 14,
 '광주시': 15,
 '구속영장': 16,
 '귀금속': 17,
 '근무': 18,
 '금은방': 19,
 '기자': 20,
 '꼬리': 21,
 '남구': 22,
 '내용': 23,
 '대응': 24,
 '대하': 25,
 '도구': 26,
 '도난': 27,
 '도둑': 28,
 '도로망': 29,
 '도망가': 30,
 '도박': 31,
 '동기': 32,
 '따라가': 33,
 '마스크': 34,
 '마치': 35,
 '만원': 36,
 '모자': 37,
 '무겁': 38,
 '바탕': 39,
 '발생': 40,
 '밝히': 41,
 '번호판': 42,
 '범인': 43,
 '범행': 44,
 '법인': 45,
 '병원': 46,
 '부서': 47,
 '부수': 48,
 '상당': 49,
 '상대로': 50,
 '상황': 51,
 '새벽': 52,
 '소유자': 53,
 '수사': 54,
 '순간': 55,
 '신고': 56,
 '신상': 57,
 '신청': 58,
 '알리': 59,
 '오전': 60,
 '운전자': 61,
 '원칙': 62,
 '월산동': 63,
 '인정': 64,
 '일시': 65,
 '임하': 66,
 '적용': 67,
 '전담': 68,
 '절도': 69,
 '조사': 70,
 '조직': 71,
 '주목': 72,
 '주인': 73,
 '준비': 74,
 '지나': 75,
 '지난달': 76,
 '지목': 77,
 '진술': 78,
 '차량': 79,
 '착수': 80,
 '채무': 81,
 '체포': 82,
 '초급': 83,
 '침입자': 84,
 '특수': 85,
 '특정인': 86,
 '파악': 87,
 '파출소': 88,

In [91]:
inv_vocab = {v: k for k, v in tfidf_vectorizer.vocabulary_.items()}
inv_vocab

{0: '가리',
 1: '간부',
 2: '검거',
 3: '경안',
 4: '경위',
 5: '경찰',
 6: '경찰관',
 7: '계급',
 8: '곳곳',
 9: '관내',
 10: '관련',
 11: '관용',
 12: '광주광역시',
 13: '광주남부경찰서',
 14: '광주서부경찰서',
 15: '광주시',
 16: '구속영장',
 17: '귀금속',
 18: '근무',
 19: '금은방',
 20: '기자',
 21: '꼬리',
 22: '남구',
 23: '내용',
 24: '대응',
 25: '대하',
 26: '도구',
 27: '도난',
 28: '도둑',
 29: '도로망',
 30: '도망가',
 31: '도박',
 32: '동기',
 33: '따라가',
 34: '마스크',
 35: '마치',
 36: '만원',
 37: '모자',
 38: '무겁',
 39: '바탕',
 40: '발생',
 41: '밝히',
 42: '번호판',
 43: '범인',
 44: '범행',
 45: '법인',
 46: '병원',
 47: '부서',
 48: '부수',
 49: '상당',
 50: '상대로',
 51: '상황',
 52: '새벽',
 53: '소유자',
 54: '수사',
 55: '순간',
 56: '신고',
 57: '신상',
 58: '신청',
 59: '알리',
 60: '오전',
 61: '운전자',
 62: '원칙',
 63: '월산동',
 64: '인정',
 65: '일시',
 66: '임하',
 67: '적용',
 68: '전담',
 69: '절도',
 70: '조사',
 71: '조직',
 72: '주목',
 73: '주인',
 74: '준비',
 75: '지나',
 76: '지난달',
 77: '지목',
 78: '진술',
 79: '차량',
 80: '착수',
 81: '채무',
 82: '체포',
 83: '초급',
 84: '침입자',
 85: '특수',
 86: '특정인',
 87: '파악',
 88: '파출소',

In [92]:
dictionary = tfidf_vectorizer.get_feature_names()

for vec in tfidf_matrix.toarray():
    vector = [[inv_vocab[idx], np.around(tfidf,decimals=2)] for idx, tfidf in enumerate(vec) if tfidf > 0]
    print(vector)


[['광주광역시', 0.33], ['귀금속', 0.33], ['금은방', 0.27], ['남구', 0.33], ['도난', 0.25], ['만원', 0.33], ['상당', 0.33], ['새벽', 0.33], ['월산동', 0.33], ['지나', 0.3]]
[['간부', 0.41], ['경찰', 0.57], ['범인', 0.46], ['초급', 0.41], ['현직', 0.37]]
[['광주남부경찰서', 0.32], ['금은방', 0.29], ['도난', 0.27], ['수사', 0.32], ['신고', 0.36], ['전담', 0.36], ['조직', 0.36], ['주인', 0.36], ['착수', 0.36]]
[['도난', 0.64], ['지나', 0.77]]
[['광주시', 0.75], ['병원', 0.66]]
[['경찰', 0.28], ['법인', 0.45], ['병원', 0.4], ['신상', 0.45], ['체포', 0.4], ['파악', 0.45]]
[['검거', 0.44], ['경찰', 0.55], ['도난', 0.33], ['발생', 0.44], ['상황', 0.44]]
[['경위', 0.52], ['경찰관', 0.52], ['계급', 0.52], ['현직', 0.42]]
[['간부', 0.71], ['초급', 0.71]]
[['관내', 0.3], ['광주서부경찰서', 0.34], ['근무', 0.34], ['도둑', 0.6], ['순간', 0.34], ['체포', 0.3], ['파출소', 0.34]]
[['범행', 0.42], ['오전', 0.52], ['일시', 0.52], ['지난달', 0.52]]
[['마스크', 0.71], ['모자', 0.71]]
[['금은방', 0.37], ['도구', 0.46], ['부수', 0.46], ['임하', 0.46], ['준비', 0.46]]
[['가리', 0.46], ['도망가', 0.46], ['번호판', 0.46], ['차량', 0.37], ['침입자', 0.46]]
[['경찰', 0.52],

#### **sklearn의 TfidfTransformer를 이용한 TF_IDF 추출**


In [94]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 
import pandas as pd

sentenceIter = RawTextReader(proc_article_path)
docs = list(sentenceIter)

cv=CountVectorizer() 
word_count_vector=cv.fit_transform(docs)

tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
경찰,2.216395
도난,2.686399
금은방,2.909543
현직,2.909543
차량,2.909543
...,...
동기,3.602690
도박,3.602690
도망가,3.602690
무겁,3.602690


In [95]:
# count matrix 
count_vector=cv.transform(docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 

#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
새벽,0.333411
광주광역시,0.333411
남구,0.333411
귀금속,0.333411
만원,0.333411
...,...
동기,0.000000
도박,0.000000
도망가,0.000000
도로망,0.000000
